# 리뷰 데이터 작업

In [ ]:
import pandas as pd
import os
import glob
import numpy as np

In [ ]:
place_list = pd.read_excel('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/관광지 목록 수정본.xlsx')
place_list.drop(['NO'], axis=1, inplace=True)

#전체 파일 불러오기
def load_csv_files_naver(dir):
    files = os.listdir(dir)
    csv_files = [file for file in files if file.endswith('.csv')] #파일 이름 불러오기

    dataframes = {}
    for file in csv_files:
        dataframe_name = os.path.splitext(file)[0]  # 파일 이름에서 확장자 제거
        data = pd.read_csv(os.path.join(dir, file))
        data = list(data.iloc[:,1])
        dataframes[dataframe_name] = data

    return dataframes

def load_csv_files_trip(dir):
    files = os.listdir(dir)
    csv_files = [file for file in files if file.endswith('.csv')] #파일 이름 불러오기

    dataframes = {}
    for file in csv_files:
        dataframe_name = os.path.splitext(file)[0]  # 파일 이름에서 확장자 제거
        data = pd.read_csv(os.path.join(dir, file))
        data = list(data.iloc[:,4])
        dataframes[dataframe_name] = data

    return dataframes


dir_naver = '/content/drive/MyDrive/DScover/2023-2 메인프/데이터/리뷰 area code명/네이버 리뷰'
dir_trip = '/content/drive/MyDrive/DScover/2023-2 메인프/데이터/리뷰 area code명/트립어드바이저 크롤링'
result_naver = load_csv_files_naver(dir_naver)
result_trip = load_csv_files_trip(dir_trip)

In [ ]:
merged_dict = {**result_naver, **result_trip}

In [ ]:
merged_dict.keys()

In [ ]:
# key값 중 ddp 키값 수정
merged_dict['POI083'] = merged_dict.pop('동대문디자인플라자(트립이랑 합치기)')

In [ ]:
# 다시 key값 체크 > 맞음
merged_dict.keys()

dict_keys(['POI032', 'POI110', 'POI111', 'POI112', 'POI108', 'POI113', 'POI109', 'POI103', 'POI105', 'POI102', 'POI101', 'POI099', 'POI091', 'POI011', 'POI087', 'POI095', 'POI094', 'POI096', 'POI098-2', 'POI097', 'POI093', 'POI098', 'POI092', 'POI008', 'POI012-2', 'POI009', 'POI001-2', 'POI001', 'POI002', 'POI003', 'POI004', 'POI005', 'POI005-3', 'POI005-2', 'POI005-4', 'POI060', 'POI055', 'POI009-2', 'POI010', 'POI012', 'POI017', 'POI034', 'POI038', 'POI040', 'POI045', 'POI046', 'POI054', 'POI057', 'POI058', 'POI059', 'POI062', 'POI063', 'POI064', 'POI066', 'POI067', 'POI073', 'POI074', 'POI078', 'POI081', 'POI082', 'POI083', 'POI084', 'POI086', 'POI088', 'POI090', 'POI104', 'POI100', 'POI014'])

In [ ]:
merged_dict.values()

In [ ]:
# 데이터 예시 보기 : key값이 "POI102" 일때
merged_dict['POI102']

### 리뷰 데이터 딕셔너리 생성

In [ ]:
# 각 관광지 ID별 리뷰 데이터프레임을 저장할 딕셔너리
df_dict = {}

# merged_dict 내의 각 관광지 ID와 리뷰에 대한 처리
for key, reviews in merged_dict.items():
    # 리스트를 DataFrame으로 변환하며, 열 이름을 'content'로 지정
    reviews_df = pd.DataFrame(reviews, columns=['content'])

    # content 열을 문자열로 변환
    reviews_df['content'] = reviews_df['content'].astype(str)

    # 중복 제거, 한글 처리, 띄어쓰기 조정
    reviews_df.drop_duplicates(inplace=True)
    reviews_df['content'] = reviews_df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
    reviews_df['content'] = reviews_df['content'].str.replace(r"\s+", " ", regex=True)  # 연속된 공백을 하나의 공백으로 대체
    reviews_df['content'] = reviews_df['content'].str.strip()  # 문자열 앞뒤의 공백 제거

    # 빈 문자열 제거
    reviews_df['content'].replace('', pd.NA, inplace=True)  # 빈 문자열을 NA 값으로 변경
    reviews_df.dropna(inplace=True)  # NA 값이 있는 행 제거

    # 전처리된 데이터프레임을 df_dict에 저장
    df_dict[key] = reviews_df

In [ ]:
# 예시: POI102에 대한 데이터프레임 확인
print(df_dict['POI102'].head())

In [ ]:
# 리뷰 수 전처리를 위한 boxplot 설정
import matplotlib.pyplot as plt

# df_dict에서 각 데이터프레임의 행 수를 리스트로 추출
rows_df = [len(df) for df in df_dict.values()]

# 박스플롯 생성
plt.figure(figsize=(10, 6))
plt.boxplot(rows_df)
plt.title("리뷰들의 개수 분포")
plt.ylabel("리뷰개수")
plt.show()

In [ ]:
# 통계값 수치로 표현
median = np.median(rows_df)  # 중앙값
mean = np.mean(rows_df)      # 평균
q1 = np.percentile(rows_df, 25)  # 제1사분위수 (Q1)
q3 = np.percentile(rows_df, 75)  # 제3사분위수 (Q3)

print(median, mean, q1, q3)

# 평균인 290개의 리뷰만 사용하기로 결정

106.0 290.52941176470586 31.75 288.25


In [ ]:
# 리뷰 수 줄이는 전처리
# 임베딩에 사용할 최종 딕셔너리 생성
final_dict = {}

for key, reviews_df in df_dict.items():
    if len(reviews_df) <= 290:
        # 데이터프레임의 크기가 290 이하인 경우 모든 행을 사용
        final_dict[key] = reviews_df
    else:
        # 데이터프레임의 크기가 290보다 큰 경우 무작위로 290개의 행을 추출
        final_dict[key] = reviews_df.sample(n=290, random_state=1234)

In [ ]:
final_dict

In [ ]:
# final_dict를 json 파일로 저장
import json

# 데이터프레임을 JSON 형식으로 변환하여 final_dict에 저장
final_dict_json = {key: df.to_json(orient='records') for key, df in final_dict.items()}

# final_dict_json을 JSON 파일로 저장
with open('final_dict.json', 'w', encoding='utf-8') as f:
    json.dump(final_dict_json, f, ensure_ascii=False)

### 리뷰 데이터 딕셔너리에 kobert 모델 적용해 임베딩 데이터 생성

In [ ]:
# JSON 파일 로드
with open('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/final_dict.json', 'r', encoding='utf-8') as f:
    loaded_dict_json = json.load(f)

# JSON을 다시 데이터프레임으로 변환
loaded_dict = {key: pd.read_json(json_str, orient='records') for key, json_str in loaded_dict_json.items()}

In [ ]:
# 데이터 구조 확인
loaded_dict

In [ ]:
# 임베딩 함수 설정
# 처리 속도에 따라 max_length와 batch_size 조정 중
def create_kobert_embeddings(reviews_df, tokenizer, bertmodel, max_length=512, batch_size=32):
    # GPU 사용 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bertmodel.to(device)

    # 토큰화 및 정수 인덱스 변환 함수
    def tokenize_data(text):
        return tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )

    # DataLoader 설정
    data_loader = DataLoader(reviews_df['content'].tolist(), batch_size=batch_size)  # 수정된 부분

    # 임베딩 벡터를 저장할 리스트
    embeddings = []

    # 데이터 로더를 통해 배치 단위로 임베딩 생성
    bertmodel.eval()
    with torch.no_grad():
        for texts in data_loader:
            inputs = [tokenize_data(text) for text in texts]
            input_ids = torch.cat([x['input_ids'] for x in inputs], dim=0).to(device)
            attention_mask = torch.cat([x['attention_mask'] for x in inputs], dim=0).to(device)

            outputs = bertmodel(input_ids, attention_mask=attention_mask)
            batch_embeddings = outputs[0].mean(dim=1).cpu().numpy()  # CLS 토큰의 임베딩 사용
            embeddings.extend(batch_embeddings)

    return np.array(embeddings)

In [ ]:
# 각 관광지별 리뷰 데이터에 대한 임베딩 생성
# 경과 속도를 알아볼 tdqm 라이브러리 import
from tqdm import tqdm

# 임베딩 결과값 담을 딕셔너리 생성
embeddings_dict = {}

# 리뷰값이 비어있는 관광지(POI104) 제거
del loaded_dict['POI014']

# 임베딩 생성 반복문 실행
for key in tqdm(loaded_dict.keys(), desc="Processing"):
    reviews_df = loaded_dict[key]
    embeddings = create_kobert_embeddings(reviews_df, tokenizer, bertmodel)
    embeddings_dict[key] = embeddings

Processing: 100%|██████████| 67/67 [05:06<00:00,  4.58s/it]


In [ ]:
# 예시 : POI102에 대한 임베딩결과 확인
print(embeddings_dict['POI102'])

[[ 0.08602966 -0.35273752 -0.12926638 ... -0.11971171 -0.22522311
   0.10809118]
 [ 0.282117   -0.4027114   0.19481839 ... -0.27549416 -0.11033238
   0.03138701]
 [ 0.01652298 -0.246156    0.1363411  ... -0.0901429  -0.34894508
   0.07066087]
 ...
 [ 0.16724809 -0.33610344 -0.06946187 ... -0.25019246 -0.21871768
   0.21711633]
 [ 0.32292023 -0.41144797 -0.04827482 ... -0.01992172 -0.16827008
   0.23612022]
 [ 0.3087889  -0.3713088  -0.0784919  ... -0.33274195 -0.2826385
  -0.25335154]]


In [ ]:
# 임베딩 벡터 json 파일로 저장

# 임베딩 벡터를 리스트로 변환하여 JSON 저장 가능한 형태로 만듦
embeddings_dict_json = {key: embeddings.tolist() for key, embeddings in embeddings_dict.items()}

# JSON 파일로 저장
with open('embeddings_dict_reviews.json', 'w', encoding='utf-8') as f:
    json.dump(embeddings_dict_json, f, ensure_ascii=False)

In [ ]:
# 런타임 재배정 시, 임베딩값 파일 불러오기
import json

with open('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/embeddings_dict_reviews.json', 'r', encoding='utf-8') as f:
    loaded_embeddings_dict_json = json.load(f)

# 리스트를 다시 Numpy 배열로 변환
loaded_embeddings_dict = {key: np.array(embeddings) for key, embeddings in loaded_embeddings_dict_json.items()}

In [ ]:
# 데이터 불러오기가 적절한지 확인
loaded_embeddings_dict